In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from dgl.nn import EGATConv, EdgeWeightNorm, GraphConv
from dgl.utils import expand_as_pair
import time

In [9]:
from config import Config
class Transformer(nn.Module):
    '''
    transformer encoder for reservoir readout layer
    '''
    def __init__(self, config:Config) -> None:
        super(Transformer, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=config.d_model, 
                                                   nhead=config.n_heads,
                                                   dim_feedforward=config.d_ff,)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=config.encoder_layer)
        self.avgpool = nn.AdaptiveAvgPool1d(output_size=1)
        self.fc = nn.Linear(config.N_hid, config.d_model)
    
    def forward(self, x):
        '''
        input x shape = [batch, channel, N_hid]
        channel = config.frames
        '''
        x = self.fc(x)                  # out shape [batch, channel, d_model]
        x = self.transformer_encoder(x) # out shape [batch, channel, d_model]
        x = self.avgpool(x)             # out shape [batch, channel, 1]
        x = torch.flatten(x, 1)         # out shape [batch, channel]
        return x

In [31]:
configuration = Config()
configuration.d_model = 64
configuration.encoder_layer = 2
configuration.n_heads = 4
configuration.d_ff = 64

configuration.lr = 1e-3
configuration.epoch = 3000
model = Transformer(configuration)
# sum(p.numel() for p in model.parameters() if p.requires_grad)

def train_transformer_readout(model:Transformer,
                      config:Config,
                      X_train,
                      X_test,
                      y_train,
                      y_test,
                      ):
    train_num = X_train.shape[0]
    test_num = X_test.shape[0]
    iteration = int(train_num/config.batch_size)
    iter = int(test_num/config.batch_size)
    cost = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    for epoch in range(config.epoch) :
        model.train()
        sum_loss = 0
        train_correct = 0
        for i in range(iteration):
            x = X_train[i*config.batch_size:(i+1)*config.batch_size] # [batch, config.frames, config.N_hid]
            y = y_train[i*config.batch_size:(i+1)*config.batch_size]
            print(y.shape)
            out = model(x)

            optimizer.zero_grad()
            loss = cost(out, y)
            loss.backward()
            optimizer.step()
    
            _, id = torch.max(out.data, 1)
            sum_loss += loss.data
            train_correct+=torch.sum(id==y.data)
        
        model.eval()
        test_correct = 0
        for i in range(iter):
            x = X_test[i*config.batch_size:(i+1)*config.batch_size]
            y = y_test[i*config.batch_size:(i+1)*config.batch_size]
            outputs = model(x)
            _, id = torch.max(outputs.data, 1)
            test_correct += torch.sum(id == y.data)
        if config.verbose:
            print('[%d,%d] loss:%.03f, train acc:%.4f, test acc:%.4f' % (epoch+1, config.epoch, sum_loss/iteration, train_correct/train_num, test_correct/test_num))
        
    return train_correct / train_num, test_correct / test_num

In [32]:
X_train = train_rs[:,1:,0:config.N_hid]
y_train = train_label
X_test = test_rs[:,1:,0:config.N_hid]
y_test = test_label

train_num = X_train.shape[0]
test_num = X_test.shape[0]
iteration = int(train_num/config.batch_size)
iter = int(test_num/config.batch_size)
cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=configuration.lr)
for epoch in range(configuration.epoch) :
    model.train()
    sum_loss = 0
    train_correct = 0
    for i in range(iteration):
        x = X_train[i*config.batch_size:(i+1)*config.batch_size] # [batch, config.frames, config.N_hid]
        y = y_train[i*config.batch_size:(i+1)*config.batch_size]
        out = model(x)

        optimizer.zero_grad()
        loss = cost(out, y)
        loss.backward()
        optimizer.step()

        _, id = torch.max(out.data, 1)
        sum_loss += loss.data
        train_correct+=torch.sum(id==y.data)
    
    model.eval()
    test_correct = 0
    for i in range(iter):
        x = X_test[i*config.batch_size:(i+1)*config.batch_size]
        y = y_test[i*config.batch_size:(i+1)*config.batch_size]
        outputs = model(x)
        _, id = torch.max(outputs.data, 1)
        test_correct += torch.sum(id == y.data)
    if config.verbose:
        print('[%d,%d] loss:%.03f, train acc:%.4f, test acc:%.4f' % (epoch+1, configuration.epoch, sum_loss/iteration, train_correct/train_num, test_correct/test_num))

[1,300] loss:3.392, train acc:0.1160, test acc:0.1140
[2,300] loss:3.365, train acc:0.1150, test acc:0.1050
[3,300] loss:3.341, train acc:0.1000, test acc:0.1180
[4,300] loss:3.314, train acc:0.0960, test acc:0.1250
[5,300] loss:3.297, train acc:0.1040, test acc:0.0970
[6,300] loss:3.265, train acc:0.1340, test acc:0.1180
[7,300] loss:3.242, train acc:0.1130, test acc:0.0910
[8,300] loss:3.213, train acc:0.1450, test acc:0.1030
[9,300] loss:3.189, train acc:0.1260, test acc:0.0860
[10,300] loss:3.164, train acc:0.1240, test acc:0.1190
[11,300] loss:3.147, train acc:0.1270, test acc:0.1070
[12,300] loss:3.114, train acc:0.1190, test acc:0.0980
[13,300] loss:3.121, train acc:0.1080, test acc:0.1040
[14,300] loss:3.078, train acc:0.1270, test acc:0.1130
[15,300] loss:3.052, train acc:0.1240, test acc:0.1010
[16,300] loss:3.033, train acc:0.1240, test acc:0.1360
[17,300] loss:3.022, train acc:0.1230, test acc:0.0790
[18,300] loss:3.032, train acc:0.1430, test acc:0.1240
[19,300] loss:3.021

KeyboardInterrupt: 

In [22]:
train_transformer_readout(model,configuration, train_rs[:,1:,0:config.N_hid], train_label, test_rs[:,1:,0:config.N_hid], test_label)

torch.Size([50, 30, 1000])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [39]:
for i in avgpool.named_parameters():
    print(i[1].shape)

In [13]:
from train_gpu import inference_new
run_time = time.strftime("%Y.%m.%d-%H-%M-%S", time.localtime())
# param_search(run_time)
from config import Config
from data import part_DATA
from RC import torchRC, EGAT, EGCN
import dgl
from RC import MLP

config = Config()
config.device = 'cpu'
config.data = 'mnist'
config.train_num = 1000
config.test_num = 1000
config.N_in = 28*28
config.N_out = 10
train_loader, test_loader = part_DATA(config)

model = torchRC(config).to(config.device)
train_rs, train_label = inference_new(model, config, train_loader,)
test_rs, test_label = inference_new(model, config, test_loader,)


Data Generation Finish
finish inference
finish inference


In [14]:
train_rs.shape

torch.Size([1000, 31, 2000])

In [20]:
from train_gpu import train_mlp_readout
config.epoch = 300

# reduce frame dimension
train_rs = train_rs[:,1:,:].mean(1)
test_rs = test_rs[:,1:,:].mean(1)

mlp = MLP(2*config.N_hid, config.mlp_hid, config.N_out).to(model.device)
train_score, test_score, = train_mlp_readout(model=mlp, 
                                            config=config,
                                            X_train=train_rs,
                                            X_test=test_rs,
                                            y_train=train_label,
                                            y_test=test_label)

[1,300] loss:1.843, train acc:0.4879, test acc:0.6850
[2,300] loss:1.175, train acc:0.7510, test acc:0.7850
[3,300] loss:0.856, train acc:0.8098, test acc:0.8200
[4,300] loss:0.688, train acc:0.8372, test acc:0.8465
[5,300] loss:0.593, train acc:0.8537, test acc:0.8595
[6,300] loss:0.532, train acc:0.8639, test acc:0.8715
[7,300] loss:0.489, train acc:0.8716, test acc:0.8770
[8,300] loss:0.456, train acc:0.8772, test acc:0.8835
[9,300] loss:0.430, train acc:0.8825, test acc:0.8865
[10,300] loss:0.408, train acc:0.8875, test acc:0.8920
[11,300] loss:0.390, train acc:0.8912, test acc:0.8930
[12,300] loss:0.375, train acc:0.8948, test acc:0.8945
[13,300] loss:0.360, train acc:0.8995, test acc:0.8970
[14,300] loss:0.348, train acc:0.9036, test acc:0.8990
[15,300] loss:0.336, train acc:0.9064, test acc:0.9010
[16,300] loss:0.327, train acc:0.9096, test acc:0.9030
[17,300] loss:0.318, train acc:0.9125, test acc:0.9065
[18,300] loss:0.309, train acc:0.9152, test acc:0.9070
[19,300] loss:0.302

In [61]:
v = train_rs[index][:,1:,0:config.N_hid]
# v = v.transpose(1,2).view(-1,30)
# node_feat = Egcn(g, v[0:config.N_hid], edge_attr)

In [76]:
train_label[index].view(1,-1)[0]

tensor([0, 3, 4, 7, 0])

In [69]:
v.shape

torch.Size([5, 30, 200])

In [70]:
batch_v = v.transpose(1,2).reshape(-1, 30)
batch_g = dgl.batch([g]*5)
batch_edge_attr = torch.cat(([edge_attr]*5))

In [71]:
node_feat = Egcn(batch_g, batch_v, batch_edge_attr)
# pred = node_feat.argmax(1)

In [73]:
batch_label

NameError: name 'batch_label' is not defined

In [65]:
pred

tensor([5, 8, 5, 5, 9, 9, 9, 8, 8, 9])

In [20]:
for i in transformer_encoder.named_parameters():
    print(i[1].shape)

torch.Size([1536, 512])
torch.Size([1536])
torch.Size([512, 512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1536, 512])
torch.Size([1536])
torch.Size([512, 512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1536, 512])
torch.Size([1536])
torch.Size([512, 512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1536, 512])
torch.Size([1536])
torch.Size([512, 512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1536, 512])
torch.Size([

In [78]:
64*30+64+64*32*32+310+330

68160

In [18]:
batch_f.shape

torch.Size([12, 30])

In [22]:
u = [0,1,2,3,2,5]
v = [1,2,3,4,0,3]
g = dgl.graph((u,v))
g = dgl.add_self_loop(g)

batch_g = dgl.batch([g, g])
feat = torch.ones(6, 3)
batch_f = torch.cat([feat, feat])

edge_weight = torch.tensor([0.5, 0.6, 0.4, 0.7, 0.9, 0.1, 1, 1, 1, 1, 1, 1])
norm = EdgeWeightNorm(norm='both')
norm_edge_weight = norm(g, edge_weight)
conv1 = GraphConv(3, 2, norm='none', weight=True, bias=True, activation=None)
conv2 = GraphConv(3, 2, norm='none', weight=True, bias=True, activation=nn.Softmax())
conv2.weight = conv1.weight
conv2.bias = conv1.bias

# out1 = conv(batch_g, batch_f, edge_weight=norm_edge_weight)
out2 = conv1(batch_g, batch_f, edge_weight=None)
out3 = conv2(batch_g, batch_f, edge_weight=None)
# out3 = conv(g, feat, edge_weight=edge_weight)
# print(out1,'\n', out2, '\n', out3)

c:\ProgramData\Anaconda3\envs\snn\lib\site-packages\dgl\nn\pytorch\conv\graphconv.py:447: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rst = self._activation(rst)


In [23]:
out2,out3

(tensor([[-3.1475,  0.8163],
         [-3.1475,  0.8163],
         [-3.1475,  0.8163],
         [-4.7213,  1.2245],
         [-3.1475,  0.8163],
         [-1.5738,  0.4082],
         [-3.1475,  0.8163],
         [-3.1475,  0.8163],
         [-3.1475,  0.8163],
         [-4.7213,  1.2245],
         [-3.1475,  0.8163],
         [-1.5738,  0.4082]], grad_fn=<AddBackward0>),
 tensor([[0.0186, 0.9814],
         [0.0186, 0.9814],
         [0.0186, 0.9814],
         [0.0026, 0.9974],
         [0.0186, 0.9814],
         [0.1211, 0.8789],
         [0.0186, 0.9814],
         [0.0186, 0.9814],
         [0.0186, 0.9814],
         [0.0026, 0.9974],
         [0.0186, 0.9814],
         [0.1211, 0.8789]], grad_fn=<SoftmaxBackward0>))

In [23]:
a = graph.adjacency_matrix()

TypeError: can't convert Sparse layout tensor to numpy.convert the tensor to a strided layout first.

In [24]:
num_nodes, num_edges = 8, 30
node_dim = 20
edge_dim = 1
graph = dgl.rand_graph(num_nodes, num_edges)
node_feats = torch.rand((num_nodes, node_dim))
edge_feats = torch.rand((num_edges, edge_dim))
egat = EGATConv(in_node_feats=node_dim,
                in_edge_feats=edge_dim,
                out_node_feats=15,
                out_edge_feats=1,
                num_heads=3)
#forward pass
new_node_feats, new_edge_feats = egat(graph, node_feats, edge_feats)
new_node_feats.shape, new_edge_feats.shape

(torch.Size([8, 3, 15]), torch.Size([30, 3, 1]))

In [28]:
new_edge_feats[0]

tensor([[1.7398],
        [2.1219],
        [0.2601]], grad_fn=<SelectBackward0>)

In [29]:
new_edge_feats.mean(1)[0]

tensor([1.3739], grad_fn=<SelectBackward0>)

In [47]:
import dgl.data

dataset = dgl.data.CoraGraphDataset('./data')
print('Number of categories:', dataset.num_classes)
g = dataset[0]
print(g.ndata)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}


In [43]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [13]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(30):
        logits = model(g, features)
        pred = logits.argmax(1)
        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

NameError: name 'GCN' is not defined

In [7]:
from config import Config as config
from RC import torchRC
model = torchRC(config)
mem, spike = model(torch.rand(16, config.frames, 50))
print(mem.shape)

torch.Size([1, 16, 101, 200])


In [45]:
A = model.reservoir.A.numpy()
edge_index = torch.tensor(np.where(A!=0), dtype=torch.long)
edge_attr = torch.tensor(np.array([A[i,j] for i,j in edge_index.T]))
u = edge_index[0]
v = edge_index[1]
mems = mem[0,0,1:].T
g = dgl.graph((u, v))
g.ndata['x'] = mems
g.edata['w'] = edge_attr

h = dgl.graph((u, v))
h.ndata['x'] = mems

In [40]:
g = dgl.graph((u, v))
g.ndata['x'] = mems
g.edata['w'] = edge_attr

h = dgl.graph((u, v))
h.ndata['x'] = mems

In [37]:
from dgl.nn import GraphConv
conv1 = GraphConv(config.frames, 16)
out = conv1(g, mems)
out_h = conv1(h, mems)
print(out_h[0])
print(out[0])

In [47]:
edge_attr

tensor([ 0.3371, -0.0486,  0.0422,  ...,  0.0489, -0.0459,  0.3718])

In [46]:
from torch_geometric.nn import GCNConv
conv2 = GCNConv(100, 16)
conv2(mems, edge_index, edge_attr)

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<AddBackward0>)